### Installation

In [1]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

### Unsloth

In [3]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit", # Llama 3.2 vision support
    "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
    "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit", # Can fit in a 80GB card!
    "unsloth/Llama-3.2-90B-Vision-bnb-4bit",

    "unsloth/Pixtral-12B-2409-bnb-4bit",              # Pixtral fits in 16GB!
    "unsloth/Pixtral-12B-Base-2409-bnb-4bit",         # Pixtral base model

    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",          # Qwen2 VL support
    "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
    "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",

    "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",      # Any Llava variant works!
    "unsloth/llava-1.5-7b-hf-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/gemma-3-4b-it",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-08 03:47:03 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.2: Fast Gemma3 patching. Transformers: 4.56.2. vLLM: 0.10.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [4]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


<a name="Data"></a>
### Data Prep
AI4Math/MathVista is a data set that involves using images to solve logic and math problems, for this notebook, it will only be math problems with numeric answers for simpilicity.

In [5]:
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer
import torch

dataset = load_dataset("AI4Math/MathVista",split="testmini")

README.md: 0.00B [00:00, ?B/s]

data/testmini-00000-of-00001-725687bf7a1(…):   0%|          | 0.00/142M [00:00<?, ?B/s]

data/test-00000-of-00002-6b81bd7f7e2065e(…):   0%|          | 0.00/358M [00:00<?, ?B/s]

data/test-00001-of-00002-6a611c71596db30(…):   0%|          | 0.00/386M [00:00<?, ?B/s]

Generating testmini split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5141 [00:00<?, ? examples/s]

Let us see what our data looks like

In [6]:
dataset["decoded_image"][5]

In [7]:
dataset["question"][5], dataset["answer"][5]

('As shown in the figure, if CB = 4.0, DB = 7.0, and D is the midpoint of AC, then the length of AC is ()',
 '6cm')

The image of our data is a part of the math problem.

To make the rewarding easy later on, let us filter only the numeric answer so that we can create reward function that gives score if reward is float or not.

In [8]:
def is_numeric_answer(example):
  try:
    float(example["answer"])
    return True
  except:
    return False

dataset = dataset.filter(is_numeric_answer) #

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

We also resize the images to be 512 by 512 pixels to make the images managable in context length. We also convert them to RGB so they are compatible with TRL's trainer!

In [9]:
# Filter have big images
def resize_images(example):
    image = example["decoded_image"]
    image = image.resize((512,512))
    example["decoded_image"] = image
    return example

dataset = dataset.map(resize_images)

def convert_to_rgb(example):
    image = example["decoded_image"]
    if image.mode != "RGB":
        image = image.convert("RGB")
    example["decoded_image"] = image
    return example

dataset = dataset.map(convert_to_rgb)

Map:   0%|          | 0/566 [00:00<?, ? examples/s]

Map:   0%|          | 0/566 [00:00<?, ? examples/s]

This is the conversational template that is needed to collate the data set

In [10]:
# Define the delimiter variables for clarity and easy modification
REASONING_START = "<REASONING>"
REASONING_END = "</REASONING>"
SOLUTION_START = "<SOLUTION>"
SOLUTION_END = "</SOLUTION>"

def make_conversation(example):
    # Define placeholder constants if they are not defined globally

    # The user's text prompt
    text_content = (
        f"{example['question']}, provide your reasoning between {REASONING_START} and {REASONING_END} "
        f"and then your final answer between {SOLUTION_START} and (put a float here) {SOLUTION_END}"
    )

    # Construct the prompt in the desired multi-modal format
    prompt = [
        {
            "role": "user",
            "content": [
                {"type": "image"},  # Placeholder for the image
                {"type": "text", "text": text_content},  # The text part of the prompt
            ],
        },
    ]

    # The actual image data is kept separate for the processor
    return {"prompt": prompt, "image": example["decoded_image"], "answer": example["answer"]}

train_dataset = dataset.map(make_conversation)

#We reformatting dataset like this because decoded_images are the actual images
#The "image": example["decoded_image"] does not properly format the dataset correctly

# 1. Remove the original 'image' column
train_dataset = train_dataset.remove_columns("image")

# 2. Rename 'decoded_image' to 'image'
train_dataset = train_dataset.rename_column("decoded_image", "image")



Map:   0%|          | 0/566 [00:00<?, ? examples/s]

Applying Chat Template across the entire data set

In [11]:
train_dataset = train_dataset.map(
    lambda example: {
        "prompt": tokenizer.apply_chat_template(
            example["prompt"],
            tokenize=False,
            add_generation_prompt=False
        )
    }
)

Map:   0%|          | 0/566 [00:00<?, ? examples/s]

We use a basic formatting functions to see if reasoning starts and ends as well as if the answers were written correctly.

In [12]:
# Reward functions
def formatting_reward_func(completions,**kwargs):
    import re
    thinking_pattern = f'{REASONING_START}(.*?){REASONING_END}'
    answer_pattern = f'{SOLUTION_START}(.*?){SOLUTION_END}'

    scores=[]
    for completion in completions :
      score=0
      thinking_matches = re.findall(thinking_pattern, completion, re.DOTALL)
      answer_matches = re.findall(answer_pattern, completion, re.DOTALL)
      if len(thinking_matches) == 1 :
        score +=1.0
      if len(answer_matches) == 1 :
        score +=1.0
      scores.append(score)
    return scores


def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    import re

    answer_pattern = f'{SOLUTION_START}(.*?){SOLUTION_END}'

    responses = [re.findall(answer_pattern, completion, re.DOTALL) for completion in completions]
    q = prompts[0]

    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:{completions[0]}")
    return [2.0 if len(r)==1 and a == r[0].replace('\n','') else 0.0 for r, a in zip(responses, answer)]


Here is the first example prompt in the data set

In [13]:
train_dataset[0]["prompt"]

"<bos><start_of_turn>user\n<start_of_image>When a spring does work on an object, we cannot find the work by simply multiplying the spring force by the object's displacement. The reason is that there is no one value for the force-it changes. However, we can split the displacement up into an infinite number of tiny parts and then approximate the force in each as being constant. Integration sums the work done in all those parts. Here we use the generic result of the integration.\r\n\r\nIn Figure, a cumin canister of mass $m=0.40 \\mathrm{~kg}$ slides across a horizontal frictionless counter with speed $v=0.50 \\mathrm{~m} / \\mathrm{s}$. It then runs into and compresses a spring of spring constant $k=750 \\mathrm{~N} / \\mathrm{m}$. When the canister is momentarily stopped by the spring, by what distance $d$ is the spring compressed?, provide your reasoning between <REASONING> and </REASONING> and then your final answer between <SOLUTION> and (put a float here) </SOLUTION><end_of_turn>\n"

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [14]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 2, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = 1024,
    max_completion_length = 1024,
    #num_train_epochs = 2, # Set to 1 for a full training run
    importance_sampling_level = "sequence",
    mask_truncated_completions=False,
    loss_type='dr_grpo',
    max_steps = 60,
    save_steps = 60,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4



| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [15]:
trainer = GRPOTrainer(
    model=model,
    args=training_args,
    # Pass the processor to handle multimodal inputs
    processing_class=tokenizer,
    reward_funcs=[formatting_reward_func, correctness_reward_func],
    train_dataset=train_dataset,
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 566 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 29,802,496 of 4,329,881,968 (0.69% trained)
`generation_config` default values have been modified to match model-specific defaults: {'top_p': 0.95}. If this is not desired, please set these values explicitly.


-------------------- Question:
<bos><start_of_turn>user
<start_of_image>What is the highest value on the X axis?, provide your reasoning between <REASONING> and </REASONING> and then your final answer between <SOLUTION> and (put a float here) </SOLUTION><end_of_turn>
 
Answer:
30 
Response:

**Question:** What is the highest value on the X axis?

<REASONING>
The X axis represents "Micrograms/ml - DNP-LYSINE-HCl". The graph shows the relationship between the amount of DNP-LYSINE-HCl (in micrograms per milliliter) and the optical density. We are looking for the highest value that can be read on the X axis. The graph extends to approximately 30 micrograms per milliliter.
</REASONING>

<SOLUTION>
30</SOLUTION>
(image)
Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / formatting_reward_func / mean,rewards / formatting_reward_func / std,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std
1,0.005100,1.875000,1.207427,231.000000,119.000000,324.000000,0.000000,231.000000,119.000000,324.000000,0.001325,1.375000,0.744024,0.500000,0.925820
2,0.001300,1.375000,0.538675,282.625000,218.000000,353.000000,0.000000,282.625000,218.000000,353.000000,0.002716,1.375000,0.517549,0.000000,0.000000
3,-0.021300,1.750000,0.500000,171.125000,101.000000,240.000000,0.000000,171.125000,101.000000,240.000000,0.001615,1.750000,0.462910,0.000000,0.000000
4,0.127700,1.375000,0.478714,503.375000,163.000000,1024.000000,0.250000,329.833344,163.000000,813.000000,0.001322,1.375000,0.916125,0.000000,0.000000
5,0.000300,1.500000,0.408248,308.750000,170.000000,608.000000,0.000000,308.750000,170.000000,608.000000,0.005748,1.500000,0.755929,0.000000,0.000000
6,0.019300,1.250000,0.696923,213.625000,140.000000,296.000000,0.000000,213.625000,140.000000,296.000000,0.001513,1.250000,0.707107,0.000000,0.000000
7,-0.003200,1.375000,0.750000,189.375000,114.000000,309.000000,0.000000,189.375000,114.000000,309.000000,0.001514,1.375000,0.744024,0.000000,0.000000
8,-0.004400,1.500000,0.577350,143.125000,97.000000,225.000000,0.000000,143.125000,97.000000,225.000000,0.001702,1.500000,0.925820,0.000000,0.000000
9,-0.005900,1.625000,0.478714,333.625000,132.000000,607.000000,0.000000,333.625000,132.000000,607.000000,0.003746,1.625000,0.744024,0.000000,0.000000
10,0.022300,1.500000,0.788675,569.750000,237.000000,972.000000,0.000000,569.750000,237.000000,972.000000,0.001667,1.500000,0.755929,0.000000,0.000000


-------------------- Question:
<bos><start_of_turn>user
<start_of_image>Subtract all yellow metallic balls. Subtract all small yellow shiny things. How many objects are left?, provide your reasoning between <REASONING> and </REASONING> and then your final answer between <SOLUTION> and (put a float here) </SOLUTION><end_of_turn>
 
Answer:
8 
Response:

Let's analyze the image.

First, we need to identify all the yellow metallic balls. There are 2.
Next, we need to identify all the small yellow shiny things. There are 1.

Subtracting the yellow metallic balls from the total number of objects, we have:
Total objects = 11
Yellow metallic balls = 2
Remaining objects = 11 - 2 = 9

Now, we need to subtract the small yellow shiny things from the remaining objects.
Remaining objects = 9
Small yellow shiny things = 1
Remaining objects = 9 - 1 = 8

So, the number of objects left is 8.

<REASONING>
The image contains 11 objects: 1 large blue cube, 1 small blue cube, 1 large yellow metallic ball, 1

TrainOutput(global_step=60, training_loss=0.017263683237273616, metrics={'train_runtime': 8148.3038, 'train_samples_per_second': 0.059, 'train_steps_per_second': 0.007, 'total_flos': 0.0, 'train_loss': 0.017263683237273616})

<a name="Inference"></a>
### Inference
Let's run the model! You can modify the instruction and input—just leave the output blank.

We'll use the best hyperparameters for inference on Gemma: `top_p=0.95`, `top_k=64`, and `temperature=1.0`.

In [16]:
FastVisionModel.for_inference(model)  # Enable for inference!

image = dataset[100]["decoded_image"]
instruction = (
    f"{dataset[100]["question"]}, provide your reasoning between {REASONING_START} and {REASONING_END} "
    f"and then your final answer between {SOLUTION_START} and (put a float here) {SOLUTION_END}"
)

messages = [
    {
        "role": "user",
        "content": [{"type": "image"}, {"type": "text", "text": instruction}],
    }
]

input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
result = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                        use_cache=True, temperature = 1.0, top_p = 0.95, top_k = 64)

Okay, let's measure the nail's length to the nearest inch.

<REASONING>
We need to align the zero mark of the ruler with the beginning of the nail, and then read the point on the ruler where the end of the nail touches. Looking at the image, the nail appears to be approximately 5.5 inches long. Since we need to round to the nearest inch, we look at the half-inch mark. Since the nail's end is just slightly past the 5 inch mark, we round up to the next whole number. Therefore, the nail is approximately 6 inches long


In [17]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# processor.push_to_hub("your_name/lora_model", token = "...") # Online saving

['lora_model/processor_config.json']

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [18]:
if False:
    from unsloth import FastVisionModel

    model, processor = FastVisionModel.from_pretrained(
        model_name="lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=True,  # Set to False for 16bit LoRA
    )
    FastVisionModel.for_inference(model)  # Enable for inference!

FastVisionModel.for_inference(model)  # Enable for inference!

sample = dataset[1]
image = sample["decoded_image"].convert("RGB")
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": (
                    f"{sample["question"]}, provide your reasoning between {REASONING_START} and {REASONING_END} "
                    f"and then your final answer between {SOLUTION_START} and (put a float here) {SOLUTION_END}"
                )
            },
            {
                "type": "image",
            },
        ],
    },
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
result = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                        use_cache=True, temperature = 1.0, top_p = 0.95, top_k = 64)

Here's an analysis of the measuring cup's volume:

**REASONING**

The measuring cup is a clear plastic container with a scale marked on its side. The scale appears to be in milliliters (mL).  We need to read the volume directly from the scale.  The scale starts at 0 mL and goes up to around 750 mL.  Therefore, the total volume capacity of the measuring cup is 750 mL.

**SOLUTION**

750.0 mL<end_of_turn>


In [19]:
# Select ONLY 1 to save! (Both not needed!)

# Save locally to 16bit
if False: model.save_pretrained_merged("unsloth_finetune", tokenizer,)

# To export and save to your Hugging Face account
if False: model.push_to_hub_merged("YOUR_USERNAME/unsloth_finetune", tokenizer, token = "PUT_HERE")